In [86]:
import numpy as np
import matplotlib.pyplot as plt

a = np.array([[1, 2, 3, 4],[2, 3, 4, 5],[3, 4, 5, 6],[3, 4, 5, 6]])

b = np.array([[1,2],[3,4]])
c = np.array([[1,0],[0,1]])

A_1 = np.array([[1, 2, 3],[4, 5, 6],[7, 8, 9]])
A_2 = np.array([[0, 1, 2],[3, 4, 5],[6, 7, 8]])

A = (A_1, A_2)

B_1 = np.array([[1,2],[3,4]])
B_2 = np.array([[0,1],[2,3]])

B = (B_1, B_2)

b_conv = np.flipud(np.fliplr(b))
b_conv

B[0]

array([[1, 2],
       [3, 4]])

In [92]:
def corr(A, krnl, padding, stride):
    stride_x = stride[0]
    stride_y = stride[1]
    padding_x = padding[0]
    padding_y = padding[1]

    B = np.zeros((A.shape[0]+2*padding_x, A.shape[1]+2*padding_y))
    B[padding_x:A.shape[0]+padding_x, padding_y:A.shape[1]+padding_y] = A
    A = B


    output = np.zeros(((A.shape[0] - krnl.shape[0])//stride_x+1, (A.shape[1] - krnl.shape[1])//stride_y+1))
    for i in range(0, A.shape[0] - krnl.shape[0]+1, stride_x):
        for j in range(0, A.shape[1] - krnl.shape[1]+1, stride_y):
            output[i//stride_x,j//stride_y] = (A[i:i+krnl.shape[0],j:j+krnl.shape[1]] * krnl).sum()

    return output

def multipleChannelsCorr(A, krnl, padding, stride):

    channels = len(A)
    output_col = corr(A[0], krnl[0], padding, stride).shape[1]
    output_row = corr(A[0], krnl[0], padding, stride).shape[0]
    print(output_row, output_col)
    output = np.zeros((output_row, output_col))

    for i in range(channels):
        output += corr(A[i], krnl[i], padding, stride)

    return output

def maxPooling(A, pool_size, padding, stride):
    stride_x = stride[0]
    stride_y = stride[1]
    padding_x = padding[0]
    padding_y = padding[1]

    B = np.zeros((A.shape[1]+2*padding_x, A.shape[0]+2*padding_y))
    B[padding_x:A.shape[1]+padding_x, padding_y:A.shape[0]+padding_y] = A
    A = B

    output = np.zeros(((A.shape[0] - pool_size[0])//stride_x + 1,((A.shape[1] - pool_size[1])//stride_y + 1)))
    

    for i in range(output.shape[0]):
        for j in range(output.shape[1]):
            pool = A[i*stride_x:i*stride_x+pool_size[0],j*stride_y:j*stride_y+pool_size[1]]
            print(pool)
            output[i,j] = np.max(pool)

    return output

In [97]:
def dL_dK(dL_dO, X, padding, stride):


    padding_x = padding[0]
    padding_y = padding[1]
    
    B = np.zeros((X.shape[0]+2*padding_x, X.shape[1]+2*padding_y))
    B[padding_x:X.shape[0]+padding_x, padding_y:X.shape[1]+padding_y] = X
    X = B

    return corr(X, dL_dO, (0,0), stride)



def dL_dX(dL_dO, krnl, padding, stride):


    dL_dO_processed = np.zeros((2*(krnl.shape[0]-1) + dL_dO.shape[0] + (stride[0]-1)*(dL_dO.shape[0]-1), 2*(krnl.shape[1]-1) + dL_dO.shape[1] + (stride[1]-1)*(dL_dO.shape[1]-1)))


    for i in range(krnl.shape[0]-1, 2*(krnl.shape[0]-1) + dL_dO.shape[0] + (stride[0]-1)*(dL_dO.shape[0]-1) - 1, stride[0]):
        for j in range(krnl.shape[1]-1, 2*(krnl.shape[1]-1) + dL_dO.shape[1] + (stride[1]-1)*(dL_dO.shape[1]-1) - 1, stride[1]):
            dL_dO_processed[i,j] = dL_dO[(i-krnl.shape[0]+1)//stride[0],(j-krnl.shape[1]+1)//stride[1]]


    krnl = np.flipud(np.fliplr(krnl))


    size_0 = corr(dL_dO_processed, krnl, (0,0), (1,1)).shape[0]
    size_1 = corr(dL_dO_processed, krnl, (0,0), (1,1)).shape[1]

    output = corr(dL_dO_processed, krnl, (0,0), (1,1))[padding[0]:size_0-padding[0],padding[1]:size_1-padding[1]]   

    return output



In [99]:
print(b, "\n",a)
dL_dK(b, a, (0,0), (1,1))

[[1 2]
 [3 4]] 
 [[1 2 3 4]
 [2 3 4 5]
 [3 4 5 6]
 [3 4 5 6]]


array([[23., 33., 43.],
       [33., 43., 53.],
       [36., 46., 56.]])